# Tutorial: Uninformed Search Algorithms and CommonRoad Search

This tutorial shows how we can use motion primitives, i.e., short trajectory pieces, in uninformed search algorithms to find a trajectory that connects an **initial state** and a **goal region**.

## How to use this tutorial
Before you start with this tutorial, make sure that

* you have read through the tutorial for [CommonRoad-io](https://commonroad.in.tum.de/static/docs/commonroad-io/index.html). Its tutorial can be found [here](https://gitlab.lrz.de/tum-cps/commonroad_io/-/tree/master/commonroad%2Ftutorials). The API of CommonRoad-io can be found [here](https://commonroad.in.tum.de/static/docs/commonroad-io/api/index.html).
* you have installed all necessary modules for CommonRoad Search according to the [installation manual](https://gitlab.lrz.de/tum-cps/commonroad-search/-/blob/master/README.md).

Let's start with importing the modules we need for setting up the automaton and the CommonRoad scenario.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
sys.path.append("../../GSMP/motion_automata")
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object
from automata.HelperFunctions import load_scenario, generate_automata
from automata.helper_tree_search import *
import automata.tree_search as tree_search

## Open and Load the CR Scenario
In the next step, we load the scenario and planning problem, which we use in the uninformed search algorithm.
If you do not understand the following steps, please go back to the tutorial for [CommonRoad-io](https://commonroad.in.tum.de/tutorials/).

In [ ]:
# Load scenario ZAM_Urban-3_2
scenario_path = '../../scenarios/tutorial/'
scenario_id = 'ZAM_Tutorial_Urban-3_2'

scenario, planning_problem_set = CommonRoadFileReader(scenario_path+scenario_id+'.xml').open()

# Plot scenario and planning problem set

plt.figure(figsize=(15, 5))
draw_object(scenario)
draw_object(planning_problem_set)
plt.gca().set_aspect('equal')
plt.margins(0, 0)
plt.show()

## Generating a Maneuver Automaton

In the following, we load the motion primitives from an XML-File and generate a Maneuver Automaton.
The maneuver automaton for this tutorial consists of 7 motion primitives and stores the connectivity to other motion primitives. 

Some additional explanations on the motion primitives:
* The motion primitives are generated for the Kinematic Single Track-Model (see [Vehicle Model Documentation](https://gitlab.lrz.de/tum-cps/commonroad-vehicle-models/blob/master/vehicleModels_commonRoad.pdf)) and the vehicle parameter are chosen for a BMW320i (vehicle_type_id=2). 
* We have motion primitives for driving with constant velocity and the steering angle is changed with constant steering angle velocity. We generated motion primitives for all combinations of the steering angles in the initial state and end state for 0 rad and 0.2 rad, i.e., 4 combinations. The three motion primitives for turning left are mirrored for turning right, resulting in total 7 motion primitives. 
* motion primitives can only be connected if they have matching initial/final velocities and matching initial/final steering angles.

In [ ]:
# Load motion primitives (mp)
mp_path = 'motion_primitives_search_tutorial/'
mp_file = 'V_9.0_9.0_Vstep_0_SA_-0.2_0.2_SAstep_0.4_T_0.5_Model_BMW320i.xml'

vehicle_type_id = 2
automaton = generate_automata(vehicle_type_id, mp_file= mp_path+mp_file, search_tutorial=True)

# plot motion primitives

plt.figure(figsize=(8,8))

for mp in automaton.Primitives:
    plot_motion_primitive(mp)
plt.show()

## Breadth-First Search (BFS)

In the next step, we set-up the BFS with the generated maneuver automaton to obtain a trajectory from the initial state to the goal region. The inital state and the goal region are specified in the planning problem. 

In [ ]:
planning_problem = list(planning_problem_set.planning_problem_dict.values())[0]

# construct motion planner and set up the initial state for planning problem
bfs_planner = tree_search.BreadthFirstSearch(scenario=scenario, planningProblem=planning_problem, 
                                             automaton=automaton)

Before we run the BFS, you can check the code below to get an idea how BFS works for motion primitives.
Please note, we removed the parts for visualization in the code below so it easier to understand the code.

```python
def search_alg(self):
    '''
    Implementation of BFS (tree search) using a FIFO queue
    '''

    # First node
    initial_node = Node(path=[[self.initial_state]], primitives=[], tree_depth=0)

    # check if we already reached the goal state
    if self.reached_goal(initial_node.path[-1]):
        return self.remove_states_behind_goal(initial_node.path), initial_node.primitives

    # add current node to the frontier
    self.frontier.insert(initial_node)

    while not self.frontier.empty():
        # Pop the shallowest node
        current_node: Node = self.frontier.pop()

        # Check all possible successor primitives(i.e., actions) for current node
        for succ_primitive in current_node.get_successors():

            # translate/rotate motion primitive to current position
            current_primitive_list = copy.copy(current_node.primitives)
            path_translated = self.translate_primitive_to_current_state(succ_primitive, 
                                                                        current_node.path[-1])

            # check for collision, if is not collision free it is skipped
            if not self.check_collision_free(path_translated):
                continue

            current_primitive_list.append(succ_primitive)

            # Goal test
            if self.reached_goal(path_translated):
                path_new = current_node.path + [[current_node.path[-1][-1]] + path_translated]
                solution_path = self.remove_states_behind_goal(path_new)
                # return solution
                return self.remove_states_behind_goal(path_new), current_primitive_list

            # Inserting the child to the frontier:
            path_new = current_node.path + [[current_node.path[-1][-1]] + path_translated]
            child = Node(path=path_new, primitives=current_primitive_list, 
                         tree_depth=current_node.tree_depth + 1)
            self.frontier.insert(child)

            if path_translated[-1].time_step > self.desired_time.end:
                return None, None

    return None, None
```

Now we want to run the algorithm:

When executing the following code block, you will see a "visualize" button directly beneath the "iteration" slider if you are running this notebook for the first time. Otherwise you can always find the button on the bottom.

Click the "visualize" button and let the search algorithm run through, once it's completed, you can use the slider to see all the iterations step by step.

In [ ]:
scenario_data = (scenario, bfs_planner.initial_state, bfs_planner.egoShape, planning_problem)
display_steps(scenario_data=scenario_data, algorithm=bfs_planner.search_alg, 
              config=bfs_planner.config)

## Depth-First Search (DFS)
Now we show the same example for the DFS. We use a simple implementation of the DFS which is similar to the BFS implementation but uses a LastInFirstOut(LIFO)-queue. Since the rest of the implementation is the same as the BFS, we directly run the DFS. 

In [ ]:
# constructing the motion planner
dfs_planner = tree_search.DepthFirstSearch(scenario=scenario, planningProblem=planning_problem, 
                                          automaton=automaton)
# run the planner
display_steps(scenario_data=scenario_data, algorithm=dfs_planner.search_alg, 
              config=dfs_planner.config)

In this scenario, we were not able to find a solution using DFS, since DFS would append motion primitives for an infinitely long time (infinite state space). This shows that DFS is not complete, i.e., DFS is not guaranteed to find a solution if one exist.

To overcome this problem we introduce a depth limit, resulting in Depth-Limited Search (DLS). This search algorithm is introduced in the next section.

## Depth-Limited Search (DLS)


Before we run the algorithm, you can have a look at the impementation. We use the recursive implementation as introduced in the lecture. 

```python
def search_alg(self, limit=7):
    '''
    Recursive implementation of DLS
    '''
    
    # First node
    initial_node = Node(path=[[self.initial_state]], primitives=[], tree_depth=0)

    result = self.recursive_dls(initial_node, limit)
    if result is None:
        return None, None

    return path, list_primitives

def recursive_dls(self, current_node: Node, limit: int):

    # Goal test
    if self.reached_goal(current_node.path[-1]):
        solution_path = self.remove_states_behind_goal(current_node.path)
        
        # return solution
        return solution_path, current_node.primitives

    elif limit == 0:
        return 'cutoff'

    else:
        cutoff_occurred = False

    for succ_primitive in reversed(current_node.get_successors()):
        # translate/rotate motion primitive to current position
        current_primitive_list = copy.copy(current_node.primitives)
        path_translated = self.translate_primitive_to_current_state(succ_primitive, current_node.path[-1])

        # check for collision, if is not collision free it is skipped
        if not self.check_collision_free(path_translated):
            continue

        # Continue search with child node
        current_primitive_list.append(succ_primitive)
        path_new = current_node.path + [[current_node.path[-1][-1]] + path_translated]
        child = Node(path=path_new, primitives=current_primitive_list, 
                     tree_depth=current_node.tree_depth + 1)

        result = self.recursive_dls(current_node=child, limit=limit-1)

        if result == 'cutoff':
            cutoff_occurred = True

        elif result is not None:
            return result

    return 'cutoff' if cutoff_occurred else None

```

Now let's run the algorithm and see what changes with the introduced limit. Here we set the limit to 7, as we know from BFS there exists a solution consisting of 7 motion primtives.


In [ ]:
dls_planner = tree_search.DepthLimitedSearch(scenario=scenario, planningProblem=planning_problem, 
                                             automaton=automaton)
limit = 7
# run the planner
display_steps(scenario_data=scenario_data, algorithm=dls_planner.search_alg, 
              config=dls_planner.config, limit=7)

As you can see, depth-limited search finds a solution.

## Uniform-Cost Search

Up to now, we looked at all algorithms, which do not consider costs during search. In the following, we look at the uniform-cost search. Uniform-Cost search is optimal for any step costs, as it expands the node with the lowest path cost g(n). In this example our cost is the time to reach the goal. Thus, our cost g(n) is the time it took, to reach our current final state. 

The Uniform-Cost Search is based on the Best-First Search, which we will also use for Greedy-Best-First Search and A\* Search. These algorithms only differ in their evaluation function. In Uniform-Cost Search, the evaluation function is f(n) = g(n). 

Before we run the search, you can have a look at the implementation of the algorithm and the evaluation function. Again, we removed all the visualization parts, so it is easier to understand the code.


```python
def search_alg(self):
    '''
    Implementation of Best-First Search (tree search) using a Priority queue
    '''
    # First node
    initial_node = PrioNode(path=[[self.initial_state]], primitives=[], tree_depth=0, current_cost=0)
   

    # add current node (i.e., current path and primitives) to the frontier
    f = self.evaluation_function(initial_node)
    self.frontier.insert(item=initial_node, priority=f)

    while not self.frontier.empty():
        # Pop the shallowest node
        current_node: PrioNode = self.frontier.pop()

        # Goal test
        if self.reached_goal(current_node.path[-1]):
            solution_path = self.remove_states_behind_goal(current_node.path)
     
            # return solution
            return solution_path, current_node.primitives

        # Check all possible successor primitives(i.e., actions) for current node
        for succ_primitive in current_node.get_successors():

            # translate/rotate motion primitive to current position
            current_primitive_list = copy.copy(current_node.primitives)
            path_translated = self.translate_primitive_to_current_state(succ_primitive, 
                                                                        current_node.path[-1])
            # check for collision, if is not collision free it is skipped
            if not self.check_collision_free(path_translated):
                continue

            current_primitive_list.append(succ_primitive)

            path_new = current_node.path + [[current_node.path[-1][-1]] + path_translated]
            child_node = PrioNode(path=path_new, primitives=current_primitive_list,
                                  tree_depth=current_node.tree_depth + 1, 
                                  current_cost=current_node.current_cost)
            f = self.evaluation_function(current_node=child_node)
            print(f)

            # Inserting the child into the frontier:
            self.frontier.insert(item=child_node, priority=f)

    return None, None

def evaluation_function(self, current_node: PrioNode) -> float:
    """
    Evaluation function of UCS is f(n) = g(n)
    """
    
    if self.reached_goal(current_node.path[-1]):
        current_node.path = self.remove_states_behind_goal(current_node.path)
    # calculate g(n)
    current_node.current_cost += len(current_node.path[-1]) * self.scenario.dt

    return current_node.current_cost
```

In [ ]:
ucs_planner = tree_search.UniformCostSearch(scenario=scenario, planningProblem=planning_problem, 
                                           automaton=automaton)
# run the planner
display_steps(scenario_data=scenario_data, algorithm=ucs_planner.search_alg, 
              config=ucs_planner.config)

Congratulations! You finished the tutorial on uninformed search and commonroad search! Next, you can check out the tutorial on informed search and commonroad search.